In [ ]:
import math
import os
import pickle
import re

import matplotlib.pyplot as plt
import nltk
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn import metrics
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import (accuracy_score, classification_report,
                             confusion_matrix, precision_recall_fscore_support)
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder

In [ ]:
import nltk
nltk.download('stopwords')
  

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
default_dir = "/content/drive/MyDrive/UFS/UFS-2020.2/tcc2/data/"
data_tcc_pos_neg = default_dir+'labeled_data/dataset_label_pos_neg.csv'
data_neg_emotions = default_dir+'labeled_data/dataset_neg_emotions.csv'


In [ ]:
data = pd.read_csv(data_neg_emotions)
data['type'] = 'with_theme-' + data['sentiment']
data = data.dropna()
data.head()

,renderedContent,date,sentiment,clean_text,type
0,@CorreiosBR Já informei faz tempo.\n#raiva #ch...,2021-06-18 12:35:41+00:00,raiva,informei fazer tempo raiva chatear,with_theme-raiva
1,@MidiaNINJA @samanthaschmutz E desse modo que ...,2021-06-18 01:14:37+00:00,raiva,d esse modo muitas vezes sinto visualizar rede...,with_theme-raiva
2,a m**** deve achar demorado entrar em contato ...,2021-06-16 21:19:59+00:00,raiva,m deve achar demorar entrar contato outros dep...,with_theme-raiva
3,"Em uma propriedade, 4 animais morreram com o v...",2021-06-16 20:40:55+00:00,raiva,propriedade animais morrer vírus raivar foco d...,with_theme-raiva
4,Estudo da FGV expõe um país infeliz santosbanc...,2021-06-16 20:25:39+00:00,raiva,estudo fgv expor país infeliz santosbancarios ...,with_theme-raiva


In [ ]:
stemmer = nltk.stem.snowball.PortugueseStemmer()
analyzer = TfidfVectorizer().build_analyzer()

In [ ]:
def stemmed_words(doc):
    return (stemmer.stem(w) for w in analyzer(doc) if w[0]!='@')

In [ ]:
def create_splits(data):
    test_validation_size = int(0.20*data.shape[0])
    train, test = train_test_split(data, test_size=test_validation_size, random_state=42, stratify=data['type'])
    return train, test

  
train, test = create_splits(data)
print('Training samples:  ', train.shape[0])
print('Test samples:      ', test.shape[0])



Training samples:   61788
Test samples:       15447


In [ ]:
vectorizer = TfidfVectorizer(
    stop_words=nltk.corpus.stopwords.words('portuguese'), 
    analyzer=stemmed_words,
    min_df=0.0001, 
    max_features=100000, 
    max_df=0.8)

In [ ]:
X_train = vectorizer.fit_transform(train['clean_text'].values.astype('U'))
X_test = vectorizer.transform(test['clean_text'].values.astype('U'))

In [ ]:
# labels = {
#     'Positivo' : 0,
#     'Negativo' : 1,
# }

labels = {
    'tristeza': 0,
    'medo': 1,
    'raiva': 2,
    'desprezo' : 3,
}

y_train = train['sentiment'].map(labels).values
y_test = test['sentiment'].map(labels).values

In [ ]:
# Create Decision Tree classifer object
clf = DecisionTreeClassifier()

# Train Decision Tree Classifer
clf = clf.fit(X_train,y_train)

In [ ]:
#Predict the response for test dataset
prediction_tree = clf.predict(X_test)

In [ ]:
print(classification_report(y_test, prediction_tree))


In [ ]:
ax = sns.heatmap(confusion_matrix(y_test, prediction_tree), cmap='Greens_r', annot=True, fmt='d')
_ = ax.set(xlabel='Previsto', ylabel='Correto', title='Árvore de Decisão.')

In [ ]:
#Calculando a acurácia
acc = accuracy_score(prediction_tree, y_test)
  precision_recall_fscore = precision_recall_fscore_support(prediction_tree, y_test, average='weighted')
print(f'acc: {acc}')
print(f'\nprf: {precision_recall_fscore}')